In [ ]:
import os
import tarfile
import urllib.request

model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8.tar.gz'

model_file = model_url.split('/')[-1]  # Extract 
if not os.path.isfile(model_file):
    urllib.request.urlretrieve(model_url, model_file)

if model_file.endswith('.tar.gz'):
    tar = tarfile.open(model_file, 'r:gz')
    tar.extractall()
    tar.close()
    print(f"Extracted {model_file} successfully.")


In [ ]:
import os

def print_directory_structure(startpath, depth=3):

    prefix = '|   '
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = prefix * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        if level < depth:
            subindent = prefix * (level + 1)
            for f in files:
                print('{}{}'.format(subindent, f))

base_model_dir = '/scratch/rk42218/Building_Detection_scratch/exported_models/'
print_directory_structure(base_model_dir, depth=3)


In [ ]:
import tensorflow as tf
import os

checkpoint_dir_or_prefix = '/home/rk42218/Building_Detection/garden garden/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0.index'

if os.path.isdir(checkpoint_dir_or_prefix):
    checkpoint_prefix = tf.train.latest_checkpoint(checkpoint_dir_or_prefix)
else:
    checkpoint_prefix = checkpoint_dir_or_prefix

try:
    reader = tf.train.load_checkpoint(checkpoint_prefix)
    variable_shapes = reader.get_variable_to_shape_map()
    variable_names = sorted(variable_shapes.keys())
    
    print(f"Successfully loaded checkpoint from {checkpoint_prefix}.")
    print("Variables and their shapes:")
    for variable_name in variable_names:
        print(f"  {variable_name}: {variable_shapes[variable_name]}")
    print(f"Total variables: {len(variable_names)}")

except tf.errors.NotFoundError as e:
    print(f"Error loading checkpoint: {e}. Please check the path and try again.")


In [ ]:
import tensorflow as tf

checkpoint_path = '/home/rk42218/Building_Detection/garden garden/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0'
variables = tf.train.list_variables(checkpoint_path)

for name, shape in variables:
    print(f'{name}: {shape}')


In [ ]:
import tensorflow as tf

checkpoint_path = '/home/rk42218/Building_Detection/garden garden/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0'
variables = tf.train.list_variables(checkpoint_path)
print(variables)


In [ ]:
import tensorflow as tf

def inspect_checkpoint_tf2(checkpoint_path):
    try:
        reader = tf.train.load_checkpoint(checkpoint_path)
        
        variable_shapes = reader.get_variable_to_shape_map()
        variable_names = sorted(variable_shapes.keys())
        
        print(f"Variables in the checkpoint '{checkpoint_path}':")
        for name in variable_names:
            print(f"  {name}, shape: {variable_shapes[name]}")
        
        print("\nCheckpoint inspection completed successfully.")
    except Exception as e:
        print(f"Error loading checkpoint '{checkpoint_path}': {e}")
        print("This checkpoint may not be compatible with TensorFlow 2.")

checkpoint_prefix = '/home/rk42218/Building_Detection/garden garden/mask_rcnn_inception_resnet_v2_1024x1024_coco17_gpu-8/checkpoint/ckpt-0'

inspect_checkpoint_tf2(checkpoint_prefix)


In [ ]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")


In [ ]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
import os
import cv2
import numpy as np

def count_classes(image_folder, mask_folder):
    image_files = os.listdir(image_folder)
    mask_files = os.listdir(mask_folder)

    unique_classes = set()

    for mask_file in mask_files:
        mask_path = os.path.join(mask_folder, mask_file)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        unique_classes.update(np.unique(mask))

    return unique_classes

image_folder = "/home/rk42218/DATA_SET_1024/chunks_images/1024/"
mask_folder = "/home/rk42218/DATA_SET_1024/chunks_masks/1024_m/"

unique_classes = count_classes(image_folder, mask_folder)

print("Number of unique classes:", len(unique_classes))
print("Unique classes:", unique_classes)


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

image_folder = "/home/rk42218/DATA_SET_1024/data_split/train/images/"
mask_folder = "/home/rk42218/DATA_SET_1024/data_split/train/masks/"

def load_data(image_folder, mask_folder):
    images = []
    masks = []
    mismatches = 0  
    
    image_files = [f for f in os.listdir(image_folder) if f.startswith("chunk_") and (f.endswith(".jpg") or f.endswith(".png"))]
    
    progress_bar = tqdm(image_files, desc="Loading Images and Masks", position=0, leave=True)
    
    for filename in progress_bar:
        image_path = os.path.join(image_folder, filename)
        mask_filename = "mask_" + filename.split('.')[0] + ".png"
        mask_path = os.path.join(mask_folder, mask_filename)
        
        if os.path.isfile(mask_path):
            image = cv2.imread(image_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            
            if image is not None and mask is not None:
                images.append(image)
                masks.append(mask)
            else:
                mismatches += 1
        else:
            mismatches += 1
    
    progress_bar.close()
    
    return images, masks, mismatches

images, masks, mismatches = load_data(image_folder, mask_folder)
print(f"Loaded {len(images)} images and {len(masks)} masks with {mismatches} mismatches (missing masks or images).")

def check_classes(masks):
    unique_classes = set()
    for mask in masks:
        unique_classes.update(np.unique(mask))
    return unique_classes

unique_classes = check_classes(masks)
print("Number of classes in masks:", len(unique_classes))

binary_masks = all(set(np.unique(mask)).issubset({0, 255}) for mask in masks)
print("Masks are binary." if binary_masks else "Masks are not binary.")

In [ ]:
import tensorflow as tf

train_record_path = '/home/rk42218/DATA_SET_1024/data_split/train/train.record'

def parse_example(example):
    example_fmt = {
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/filename': tf.io.FixedLenFeature([], tf.string),
        'image/source_id': tf.io.FixedLenFeature([], tf.string),
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/format': tf.io.FixedLenFeature([], tf.string),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/text': tf.io.VarLenFeature(tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
        'image/object/difficult': tf.io.VarLenFeature(tf.int64),
        'image/object/truncated': tf.io.VarLenFeature(tf.int64),
        'image/object/view': tf.io.VarLenFeature(tf.string),
    }
    return tf.io.parse_single_example(example, example_fmt)

dataset = tf.data.TFRecordDataset(train_record_path)

for i, record in enumerate(dataset.take(1)): 
    parsed_example = parse_example(record.numpy())
    print(parsed_example)

